Biomass Grass production based on light use efficiency models

In [ ]:
install.packages("dplyr")
install.packages("sp")
install.packages("raster")
install.packages("lubridate")
install.packages("imputeTS")
install.packages("rgdal")

library(dplyr)
library(sp)
library(raster)
library(lubridate)
library(imputeTS)
library(rgdal)

In [ ]:
# Dates interval
start<-'2021-07-01'
end<-'2022-08-31'
# Thresholds depending on biome (look-up table BPLUT):
emax= 1.24   #introducir el epsilon m?ximo (g/MJ)
tmin_max= 11.39  # ºC
tmin_min= -8  # ºC
vpd_max= 3.2  # kPa
vpd_min= 0.65  # kPa 

In [ ]:
root<-getwd()  #Save root folder for the script
root

In [ ]:
  #Loading meteorological data archive, order it and interpolate
meteo<-read.csv('DHMR01.csv',dec = ',') 
meteo$fecha<-dmy(meteo$fecha) #check for other data, this order gives format to datatimes
meteo<-meteo[order(meteo$fecha),] #order
meteo<-meteo[meteo$fecha>=start&meteo$fecha<=end,]
print('Meteo archives loaded')

In [ ]:
#brick of images interpolated
NDVI_brick_interpoladas<-brick('NDVI_interpoladas_DHMR01.grd',sep='')
fechas<-data.frame(fecha=ymd(substr(names(NDVI_brick_interpoladas),5,12))) #pick up dates from available images
indices<-1:nrow(fechas) #Index from images
fechas_analisis<-fechas[fechas$fecha>=start&fechas$fecha<=end,]
indices_analisis<-indices[fechas$fecha%in%fechas_analisis]
NDVI_brick_interpoladas<-subset(NDVI_brick_interpoladas,indices_analisis)
print('loaded images')

In [ ]:
#adjust dates
fechas<-data.frame(fecha=ymd(substr(names(NDVI_brick_interpoladas),5,12))) #all dates
meteo<-merge(fechas,meteo,by='fecha',all.x=TRUE) #recalculate
print('adust dates')

In [ ]:
names(meteo)[3]<-'TMin'
names(meteo)[4]<-'TMed'
#interpolate if meteo data is missing
meteo$TMin<-round(na_interpolation(meteo$TMin),1)
meteo$TMed<-round(na_interpolation(meteo$TMed),1)
meteo$HR<-round(na_interpolation(meteo$HR),1)
meteo$Rad<-round(na_interpolation(meteo$Rad),1)
print('meteo interpolated')

In [ ]:
# ec. saturate vapor pressure
meteo$es<-(0.611*exp((17.27*meteo$TMed)/(237.3+meteo$TMed)))*10 
# ec. vapor pressure   
meteo$eamb<-meteo$es*meteo$HR/100
#in kPa
meteo$DPV<-(meteo$es-meteo$eamb)/10
  
# Scalar Tmin, scalar VPD, and maximum epsilon from BPLUT description
meteo$tminesc<-(meteo$TMin-tmin_min)/(tmin_max-tmin_min)
    if(length(meteo[meteo$tminesc<0,]$tminesc)>0) {meteo[meteo$tminesc<0,]$tminesc<-0}
    if(length(meteo[meteo$tminesc>1,]$tminesc)>0) {meteo[meteo$tminesc>1,]$tminesc<-1}
    meteo$VPDesc<-1+((vpd_min-meteo$DPV)/(vpd_max-vpd_min))
    if(length(meteo[meteo$VPDesc<0,]$VPDesc)>0){meteo[meteo$VPDesc<0,]$VPDesc<-0}
    if(length(meteo[meteo$VPDesc>1,]$VPDesc)>0){meteo[meteo$VPDesc>1,]$VPDesc<-1}
  
#epsilon
meteo$epsilon<-emax*meteo$tminesc*meteo$VPDesc
meteo$epsilong<-0.86*meteo$tminesc*meteo$VPDesc
  
#PAR from radiation
meteo$PAR<-0.48*meteo$Rad
  
#export meteo table
write.csv(meteo,file='meteo.csv')
reclass_0<-matrix(c(-Inf,0,0),ncol=3) #matriz para reclasificar todos los valores 

In [ ]:
    for (i in 1:nlayers(NDVI_brick_interpoladas)){
        imagen<-raster(NDVI_brick_interpoladas,layer=i)
        year<-substr(names(imagen),5,8)
        base<-raster(paste('verano/verano',year,'DHMR01.tif',sep=''))
        ajustada<-imagen-base   
        fpar<-1.26*ajustada-0.19
        fpar<-reclassify(fpar,reclass_0)  #ajusta a 0 los valores de fpar negativos
        if(i==1){fpar_brick<-brick(fpar)
        names(fpar_brick)[i]<-paste('fpar_',names(imagen),sep='')} 
        else{fpar_brick<-addLayer(fpar_brick,fpar)  #Lo transforma en stack pero como brick es mucho más lento
        names(fpar_brick)[i]<-paste('fpar_',names(imagen),sep='')}
        NPP<-10*fpar*meteo$PAR[i]*meteo$epsilon[i]
        NPP<-reclassify(NPP,reclass_0)  #ajusta a 0 los valores de NPP negativos          
        if(i==1){NPP_brick<-brick(NPP)
        names(NPP_brick)[i]<-paste('NPP_',names(imagen),sep='')
        acumulado<-reclassify(NPP,matrix(c(-Inf,0,0),ncol=3))
        NPPa_brick<-brick(acumulado)
        names(NPPa_brick)[i]<-paste('NPPa_',names(imagen),sep='')
                } 
        else{NPP_brick<-addLayer(NPP_brick,NPP)
             names(NPP_brick)[i]<-paste('NPP_',names(imagen),sep='')
             acumulado<-reclassify(acumulado,matrix(c(-Inf,0,0),ncol=3))
        if (substr(names(imagen),9,12)=='0831'){acumulado<-acumulado*0}else{acumulado<-acumulado+NPP}
             NPPa_brick<-addLayer(NPPa_brick,acumulado)
             names(NPPa_brick)[i]<-paste('NPPa_',names(imagen),sep='')
            }
        print(paste('layer ',i,' over ',nlayers(NDVI_brick_interpoladas),'  ',round(i*100/nlayers(NDVI_brick_interpoladas),2),'%',sep=''))
    }

In [ ]:
#All store in brick        
dir.create('biomasa')
writeRaster(fpar_brick,
            filename=paste('biomasa/fpar_emax',emax*100,'.grd',sep=''), 
            format="raster", 
            overwrite=TRUE)
writeRaster(NPP_brick, 
            filename=paste('biomasa/NPP_emax',emax*100,'.grd',sep=''), 
            format="raster", 
            overwrite=TRUE)
writeRaster(NPPa_brick, 
            filename=paste('biomasa/NPPa_emax',emax*100,'.grd',sep=''), 
            format="raster", 
            overwrite=TRUE)

In [ ]:
#Zone stats
poly<-readOGR(paste('coberturas/DHMR01.shp',sep=''))  #zone
poly<-spTransform(x = poly, #google engine proyection
                  CRSobj = '+proj=utm +zone=30 +datum=WGS84 +units=m +no_defs +ellps=WGS84 +towgs84=0,0,0 ')
NPPa_mean <- extract(NPPa_brick, poly, fun='mean', na.rm=TRUE, df=FALSE, weights = TRUE) 
NPPa_mean<-data.frame(t(NPPa_mean))  #transponemos el dataframe
colnames(NPPa_mean)<-paste(poly$Name,'_',emax*100,sep='')
NPPa_mean<-data.frame(NPPa_mean,fecha=ymd(substr(rownames(NPPa_mean),10,17)))
  
write.csv(NPPa_mean,paste('biomasa/NPPa_mean.csv',sep=''))
  
NPP_mean <- extract(NPP_brick, poly, fun='mean', na.rm=TRUE, df=FALSE, weights = TRUE) 
NPP_mean<-data.frame(t(NPP_mean))  #transponemos el dataframe
colnames(NPP_mean)<-paste(poly$Name,'_',emax*100,sep='')
NPP_mean<-data.frame(NPP_mean,fecha=ymd(substr(rownames(NPP_mean),10,17)))
  
write.csv(NPP_mean,paste('biomasa/NPP_mean.csv',sep=''))

  
setwd(root)

